In [2]:
from reachy_sdk import ReachySDK
from reachy_sdk.trajectory import goto
from reachy_sdk.trajectory.interpolation import InterpolationMode
import time

In [3]:
reachy = ReachySDK('localhost')

In [ ]:
reachy.l_arm.l_gripper.speed_limit = 80
reachy.l_arm.l_gripper.compliant = False
reachy.turn_on('l_arm')

In [ ]:
reachy.l_arm.l_gripper.goal_position = -80

In [ ]:
reachy.l_arm.l_gripper.goal_position = -5

In [4]:
reachy.turn_off('r_arm')

In [ ]:
reachy.r_arm.r_gripper.speed_limit = 80
reachy.r_arm.r_gripper.compliant = False
reachy.turn_on('r_arm')

In [ ]:
reachy.r_arm.r_gripper.goal_position = -10

In [ ]:
reachy.r_arm.r_gripper.compliant = False 
reachy.r_arm.r_gripper.goal_position = 13

In [ ]:
reachy.turn_off('r_arm')

In [ ]:
reachy.r_arm.r_gripper.registers()

In [ ]:
reachy.r_arm.r_gripper.temperature

In [ ]:
reachy.l_arm.l_gripper.temperature

In [ ]:
reachy.l_arm.l_gripper.registers()

In [ ]:
def goto_base_position(duration=1.0):
    reachy.turn_on('r_arm')
    time.sleep(0.1)
    goto(
        goal_positions=
                {reachy.r_arm.r_shoulder_pitch: 60,
                reachy.r_arm.r_shoulder_roll: -15,
                reachy.r_arm.r_arm_yaw: 0,
                reachy.r_arm.r_elbow_pitch: -95,
                reachy.r_arm.r_forearm_yaw: -15,
                reachy.r_arm.r_wrist_pitch: -50,
                reachy.r_arm.r_wrist_roll: 0},
            duration=1.0,
            interpolation_mode=InterpolationMode.MINIMUM_JERK
        )
    time.sleep(0.1)
    reachy.r_arm.r_shoulder_pitch.torque_limit = 75
    reachy.r_arm.r_elbow_pitch.torque_limit = 75

In [ ]:
import numpy as np
def goto_position(path): 
    reachy.turn_on('r_arm')
    move = np.load(path)
    move.allow_pickle=1
    listMoves = move['move'].tolist()
    listTraj = {}
    for key,val in listMoves.items():
        listTraj[eval(key)] = float(val)
    goto(
        goal_positions=listTraj, 
        duration=1.0,
        interpolation_mode=InterpolationMode.MINIMUM_JERK
    )

In [ ]:
def trajectoryPlayer(path):
    reachy.turn_on('r_arm')
    move = np.load(path)
    move.allow_pickle=1

    listMoves = move['move'].tolist()
    listTraj = [ val for key,val in listMoves.items()]
    listTraj = np.array(listTraj).T.tolist()

    sampling_frequency = 100  #en hertz
    
    recorded_joints = []
    for joint,val in listMoves.items():
        if 'neck' in joint : 
            fullName = joint
        elif 'r_' in joint: 
            fullName = joint
        elif 'l_' in joint: 
            fullName = joint
        recorded_joints.append(eval(fullName))

    for joint in recorded_joints:
        joint.compliant = False

    first_point = dict(zip(recorded_joints, listTraj[0]))
    goto(first_point, duration=2.0)

    for joints_positions in listTraj:
        for joint, pos in zip(recorded_joints, joints_positions):
            joint.goal_position = pos
        time.sleep(1 / sampling_frequency)

In [ ]:
def goto_rest_position(duration=1.0):
    time.sleep(0.1)
    reachy.head.look_at(x=1, y=0, z=0, duration=1) 
    goto_base_position(0.6 * duration)
    time.sleep(0.1)

    reachy.turn_on('r_arm')
    goto(
        goal_positions=
                {reachy.r_arm.r_shoulder_pitch: 50,
                reachy.r_arm.r_shoulder_roll: -15,
                reachy.r_arm.r_arm_yaw: 0,
                reachy.r_arm.r_elbow_pitch: -100,
                reachy.r_arm.r_forearm_yaw: -15,
                reachy.r_arm.r_wrist_pitch: -60,
                reachy.r_arm.r_wrist_roll: 0},
            duration=1.0,
            interpolation_mode=InterpolationMode.MINIMUM_JERK
        )
    time.sleep(1)

    time.sleep(0.25)


    reachy.r_arm.r_shoulder_roll.comliant = True
    reachy.r_arm.r_arm_yaw.comliant = True
    reachy.r_arm.r_elbow_pitch.comliant = True
    reachy.r_arm.r_forearm_yaw.comliant = True
    reachy.r_arm.r_wrist_pitch.comliant = True
    reachy.r_arm.r_wrist_roll.comliant = True
    reachy.r_arm.r_gripper.comliant = True

    #reachy.turn_off('r_arm')

    time.sleep(0.25)

In [ ]:
reachy.turn_off('r_arm')


In [ ]:
goto_base_position()

In [ ]:
path = "/home/reachy/repos/TicTacToe/tictactoe/movements/grab_1.npz"
trajectoryPlayer(path)

In [ ]:
goto_base_position() 
reachy.r_arm.r_gripper.goal_position = -10 #open the gripper 
#path = "/home/reachy/repos/TicTacToe/tictactoe/movements/grab_1.npz"
#goto_position(path)
path = f'/home/reachy/repos/TicTacToe/tictactoe/movements/moves_new/grab_1.npz'
trajectoryPlayer(path)
time.sleep(1)
reachy.r_arm.r_gripper.compliant = False 
reachy.r_arm.r_gripper.goal_position = 13 #close the gripper to take the cylinder 
time.sleep(1)

In [ ]:
def play_pawn(grab_index, box_index):
    reachy.r_arm.r_gripper.speed_limit = 80
    reachy.r_arm.r_gripper.compliant = False
    reachy.turn_on('r_arm')

    # Goto base position
    goto_base_position() 
    reachy.r_arm.r_gripper.goal_position = -10 #open the gripper 
    path = "/home/reachy/repos/TicTacToe/tictactoe/movements/moves-2021_right/grab_1.npz"
    goto_position(path)
    #path = f'/home/reachy/repos/TicTacToe/tictactoe/movements/moves-2021_nemo/grab_{grab_index}.npz'
    #trajectoryPlayer(path)
    time.sleep(1)
    reachy.r_arm.r_gripper.compliant = False 
    reachy.r_arm.r_gripper.goal_position = 13 #close the gripper to take the cylinder 
    time.sleep(1)


    if grab_index >= 4:
        goto(
        goal_positions = {
            reachy.r_arm.r_shoulder_pitch : reachy.r_arm.r_shoulder_pitch.goal_position+10, 
            reachy.r_arm.r_elbow_pitch : reachy.r_arm.r_elbow_pitch.goal_position+10, 
            },
            duration =1.0,
            interpolation_mode=InterpolationMode.MINIMUM_JERK
        )

    # Lift it
    path = '/home/reachy/repos/TicTacToe/tictactoe/movements/moves-2021_right/lift.npz'
    goto_position(path)
    time.sleep(0.1)

    # Put it in box_index

    path = f'/home/reachy/repos/TicTacToe/tictactoe/movements/moves-2021_right/put_{box_index}.npz'
    trajectoryPlayer(path)
    time.sleep(1)
    reachy.r_arm.r_gripper.compliant = False
    reachy.r_arm.r_gripper.goal_position = -10
    time.sleep(1)

    path = f'/home/reachy/repos/TicTacToe/tictactoe/movements/moves-2021_right/back_{box_index}_upright.npz'
    goto_position(path)


    if box_index in (8, 9):

        path = '/home/reachy/repos/TicTacToe/tictactoe/movements/moves-2021_right/back_to_back.npz'
        goto_position(path)

    path = '/home/reachy/repos/TicTacToe/tictactoe/movements/moves-2021_right/back_rest.npz'
    goto_position(path)

    goto_rest_position()

In [ ]:
path = f'/home/reachy/repos/TicTacToe/tictactoe/movements/moves-2021_nemo/put_9.npz'
trajectoryPlayer(path)

In [ ]:
play_pawn(1,1)
play_pawn(1,2)
play_pawn(1,3)
play_pawn(1,4)
play_pawn(1,5)
play_pawn(1,6)



In [ ]:
play_pawn(1,3)

In [ ]:
for f in reachy.fans.values():
    f.on()

In [1]:
reachy.turn_off('r_arm')



NameError: name 'reachy' is not defined

In [ ]:
for i in range(9):
    state = board.get_board()
    state_m  = np.where(state == 1, 3, state)
    state_m  = np.where(state_m == 2, 1, state_m)
    state_m  = np.where(state_m == 3, 2, state_m)
    game = state_m[::-1]
    game = np.array_split(state_m,3)
    print(game[0])
    print(game[1])
    print(game[2])
    print('     ')
    print('     ')

    action = - board.choose_next_action(state_m)[0] +9
    #board.play_pawn(1,action)
    time.sleep(2)
